# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [19]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [12]:
# 2) seu código aqui
sinasc.isna().sum(axis=0)

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [13]:
# 3) seu código aqui
sinasc_2 = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_2.isna().sum(axis=0)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [15]:
# 4) seu código aqui
sinasc_2.dropna(subset=["APGAR5"],inplace=True)
sinasc_2.info()
sinasc_2.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 26925 entries, 0 to 27027
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCNASC     26925 non-null  int64  
 1   IDADEMAE    26925 non-null  int64  
 2   ESTCIVMAE   26610 non-null  float64
 3   ESCMAE      26615 non-null  object 
 4   QTDFILVIVO  25359 non-null  float64
 5   GESTACAO    25709 non-null  object 
 6   GRAVIDEZ    26849 non-null  object 
 7   CONSULTAS   26925 non-null  int64  
 8   APGAR5      26925 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 2.1+ MB


C:\Users\edume\AppData\Local\Temp\ipykernel_9456\44051580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_2.dropna(subset=["APGAR5"],inplace=True)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [24]:
# 5) seu código aqui
sinasc_2[['ESTCIVMAE', 'CONSULTAS']].replace(9,np.NaN,inplace=True)


C:\Users\edume\AppData\Local\Temp\ipykernel_9456\88351418.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_2[['ESTCIVMAE', 'CONSULTAS']].replace(9,np.NaN,inplace=True)


In [23]:
# 6) Seu código aqui
sinasc_2["QTDFILVIVO"].replace(np.NaN,0,inplace=True)

In [25]:
# 7) seu código aqui
sinasc_2.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [26]:
sinasc_2['LOCNASC'].replace(9,np.NaN,inplace=True)

In [27]:
sinasc_2["GESTACAO"].replace(9,np.NaN,inplace=True)

In [28]:
sinasc_2["GRAVIDEZ"].replace(9,np.NaN,inplace=True)

In [30]:
# 8) seu código aqui
APGAR5 = sinasc_2["APGAR5"]
cond_list = [APGAR5 < 4,APGAR5 < 6, APGAR5 < 8, APGAR5 >= 8]
choice_list = ["Asfixia severa","Asfixia moderada","Asfixia leve","Normal"]
sinasc_2["APGAR5_cat"] = np.select(cond_list,choice_list)
sinasc_2

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal


In [33]:
sinasc_2["APGAR5_cat"].unique()

array(['Normal', 'Asfixia leve', 'Asfixia severa', 'Asfixia moderada'],
      dtype=object)

In [39]:
sinasc_2["ESCMAE"].dtype

dtype('O')

In [52]:
# 9) seu código aqui
sinasc_2.replace(' ', '_', regex=True,inplace=True)
sinasc_2

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_cat
0,1,19,5.0,8_a_11_anos,0.0,37_a_41_semanas,Única,4,10.0,Normal
1,1,29,2.0,8_a_11_anos,1.0,37_a_41_semanas,Única,4,9.0,Normal
2,1,37,9.0,8_a_11_anos,2.0,37_a_41_semanas,Única,4,10.0,Normal
3,1,30,5.0,12_anos_ou_mais,0.0,37_a_41_semanas,Única,3,10.0,Normal
4,1,30,2.0,8_a_11_anos,1.0,37_a_41_semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12_anos_ou_mais,1.0,32_a_36_semanas,Única,4,9.0,Normal
27024,1,19,1.0,8_a_11_anos,0.0,37_a_41_semanas,Única,4,9.0,Normal
27025,1,24,2.0,8_a_11_anos,0.0,37_a_41_semanas,Única,4,10.0,Normal
27026,1,21,2.0,8_a_11_anos,1.0,32_a_36_semanas,Única,4,9.0,Normal
